# 이미지 자르기

In [1]:
import sys
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import os
import pandas as pd

In [4]:
path_dir = '/home/adminuser/notebooks/car_classification/train'

In [5]:
category = os.listdir(path_dir)
category

['기아 레이',
 '르노삼성 QM6',
 '현대 코나 일렉트릭',
 '현대 코나',
 '현대 그랜드 스타렉스',
 '현대 LF 쏘나타',
 '르노삼성 QM3',
 '쉐보레 트랙스',
 '현대 팰리 세이드',
 '기아 셀토스',
 '쌍용 렉스턴 스포츠',
 '기아 모하비',
 '쌍용 G4 렉스턴',
 '기아 봉고3',
 '기아 K7 하이브리드 YG',
 '기아 K3 BD',
 '제네시스 G80',
 '기아 니로',
 '기아 K7 YG',
 '기아 쏘렌토',
 '기아 모닝 JA',
 '현대 그랜저 하이브리드 IG',
 '쌍용 티볼리',
 '현대 코나 하이브리드',
 '현대 그랜저 IG',
 '기아 카니발',
 '기아 K5 JF',
 '르노삼성 SM6',
 '쉐보레 스파크',
 '현대 베뉴',
 '제네시스 G90',
 '.DS_Store',
 '현대 싼타페TM']

In [11]:

# 모델 & 설정 파일
model = 'yolo_v3/yolov3.weights'
config = 'yolo_v3/yolov3.cfg'
class_labels = 'yolo_v3/coco.names'
confThreshold = 0.5
nmsThreshold = 0.4



# 네트워크 생성
    
net = cv2.dnn.readNet(model, config)

if net.empty():
    print('Net open failed!')
    sys.exit()

# 클래스 이름 불러오기

classes = []
with open(class_labels, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

colors = np.random.uniform(0, 255, size=(len(classes), 3))

# 출력 레이어 이름 받아오기

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
# output_layers = ['yolo_82', 'yolo_94', 'yolo_106']

# 실행
infos = []
createFolder('./car_cropped')

for idx,cat in enumerate(category):

    createFolder('./car_cropped/'+ cat)
    dox_dir = path_dir + '/' + cat
    files = glob.glob(dox_dir+"/*.jpg")
    for f in files:
        img = cv2.imread(f)
        head, tail = os.path.split(f)
        if img is None:
            continue

        # 블롭 생성 & 추론
        blob = cv2.dnn.blobFromImage(img, 1/255., (320, 320), swapRB=True)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # outs는 3개의 ndarray 리스트.
        # outs[0].shape=(507, 85), 13*13*3=507
        # outs[1].shape=(2028, 85), 26*26*3=2028
        # outs[2].shape=(8112, 85), 52*52*3=8112

        h, w = img.shape[:2]

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                # detection: 4(bounding box) + 1(objectness_score) + 80(class confidence)
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > confThreshold:
                    # 바운딩 박스 중심 좌표 & 박스 크기
                    cx = int(detection[0] * w)
                    cy = int(detection[1] * h)
                    bw = int(detection[2] * w)
                    bh = int(detection[3] * h)

                    # 바운딩 박스 좌상단 좌표
                    sx = int(cx - (bw / 2))
                    sy = int(cy - (bh / 2))

                    boxes.append([sx, sy, bw, bh])
                    confidences.append(float(confidence))
                    class_ids.append(int(class_id))

        # 비최대 억제
        indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)


        sx, sy, bw, bh = 0,0,0,0

        for i in indices:
            i = i[0]
            x, y, b, h = boxes[i]

            label = f'{classes[class_ids[i]]}: {confidences[i]:.2}'
            color = colors[class_ids[i]]
            if classes[class_ids[i]] == 'car':
                if bw * bh < b * h:
                    sx, sy, bw, bh = x,y,b,h

        print(sx, sy, bw, bh)
        #cv2.rectangle(img, (sx, sy, bw, bh), color, 2)

        cv2.putText(img, label, (sx, sy - 10),
                 cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)
        #crop_img = img[]

        if sx < 0 :
            sx = 0
        if sy < 0 :
            sy = 0
        crop_img = img[sy : sy+bh, sx : sx+bw]

        #plt.imshow(img)
        #plt.show()
        infos.append([tail,sx, sy, bw, bh])
        try :
            if bw * bh != 0:
                plt.imsave('./car_cropped/'+cat+'/'+tail , crop_img)
        except :
            pass


-3 50 268 108
0 31 292 99
1 43 259 100
6 59 289 103
1 21 280 134
-3 13 276 149
50 76 233 82
2 49 268 104
3 59 245 114
-7 36 307 122
-19 39 333 84
0 0 0 0
0 -1 293 160
-1 39 270 135
10 26 225 127
-9 1 315 157
24 49 230 107
-6 16 304 132
-1 47 281 100
-5 40 207 97
0 25 275 126
-2 1 206 167
-3 3 297 159
-8 47 188 104
8 47 260 102
0 38 271 133
27 58 225 98
-13 30 353 100
37 79 240 80
-5 69 285 80
-2 23 293 133
114 28 252 99
-5 8 314 146
31 57 197 87
98 38 197 78
-17 50 293 99
17 75 193 116
23 65 242 110
27 18 238 118
8 66 231 96
-11 5 333 144
-1 39 263 133
0 33 315 119
-11 -3 265 125
0 0 0 0
-3 20 301 140
12 45 267 98
-2 30 291 133
3 63 276 87
-4 4 337 135
-11 5 291 155
0 0 0 0
5 64 270 100
-11 7 334 140
1 53 284 87
11 38 275 82
-14 14 371 112
57 55 218 97
0 0 0 0
1 22 304 137
7 22 296 119
-5 10 289 139
-2 2 248 129
1 14 344 113
-5 26 292 145
30 37 210 122
-4 47 283 99
7 61 255 132
1 38 270 126
4 24 322 97
1 61 221 102
-6 9 315 131
3 37 272 120
-4 53 258 106
33 66 200 81
3 47 279 125
-8 24

In [ ]:
infos

# Train Val Test 

In [12]:
import shutil
import random
def split(img_list, test_count,val_count, train_path,test_path,val_path):
    test_files=[]
    random.seed(42)
    for i in random.sample(img_list,test_count):
        test_files.append(i)
    
    img = [x for x in img_list if x not in test_files]
    
    val_files= []
    for i in random.sample(img, val_count):
        val_files.append(i)
        
    train_files = [x for x in img_list if x not in val_files+test_files]
    
        
    for k in train_files:
        shutil.copy(k,train_path)
    
    for c in test_files:
        shutil.copy(c,test_path)
    for v in val_files:
        shutil.copy(v,val_path)

In [13]:
import shutil
import random
def split(img_list, test_count,val_count, train_path,test_path,val_path):
    test_files=[]
    random.seed(42)
    for i in random.sample(img_list,int(len(img_list)*test_count)):
        test_files.append(i)
    
    img = [x for x in img_list if x not in test_files]
    
    val_files= []
    for i in random.sample(img, int(len(img)*val_count)):
        val_files.append(i)
        
    train_files = [x for x in img_list if x not in val_files+test_files]
    
        
    for k in train_files:
        shutil.copy(k,train_path)
    
    for c in test_files:
        shutil.copy(c,test_path)
    for v in val_files:
        shutil.copy(v,val_path)

In [14]:
createFolder('./train')
createFolder('./test')
createFolder('./val')
for i in category:
    createFolder('./train/'+i)
    createFolder('./test/'+i)
    createFolder('./val/'+i)
    
    split(glob.glob('./car_cropped/'+i+'/*.jpg'), 0.2, 0.2,'./train/'+i,'./test/'+i,'./val/'+i)